In [50]:
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
#import matplotlib.pyplot as plt
import numpy as np

In [51]:
# ########################################CUSTOM DATASET ################################################################################
# ########################################CUSTOM DATASET ################################################################################
# ########################################CUSTOM DATASET ################################################################################
def one_hot_encoder(sequence):
    # Define dictionary to map nucleotides to one-hot encoding
    #nucleotide_map = {'A': [1, 0, 0, 0], 'T': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'C': [0, 0, 0, 1],'a': [1, 0, 0, 0], 't': [0, 1, 0, 0], 'g': [0, 0, 1, 0], 'c': [0, 0, 0, 1]}
    A = []
    T = []
    C = []
    G = []
    for s in sequence:
        if s.upper() == 'A':
            A.append(1)
            T.append(0)
            C.append(0)
            G.append(0)
        elif s.upper() == 'T':
            A.append(0)
            T.append(1)
            C.append(0)
            G.append(0)
        elif s.upper() == 'C':
            A.append(0)
            T.append(0)
            C.append(1)
            G.append(0)
        elif s.upper() == 'G':
            A.append(0)
            T.append(0)
            C.append(0)
            G.append(1)

        elif s.upper() == 'N':
            A.append(0)
            T.append(0)
            C.append(0)
            G.append(0)

    encoded_sequence = []
    encoded_sequence.append(A)
    encoded_sequence.append(T)
    encoded_sequence.append(C)
    encoded_sequence.append(G)
    return np.array(encoded_sequence)



In [52]:
def read_sequences_from_file(filename):
    sequences = []
    with open(filename, 'r') as file:
        for line in file:
            # Split label and sequence
            label, sequence = line.strip().split(' ', 1)
            sequences.append((int(label), sequence))
    return sequences

In [53]:
def process_sequences(filename):
    sequences = read_sequences_from_file(filename)
    #c = 0
    processed_data = []
    for label, sequence in sequences:
        # One-hot encode the sequence
        one_hot_encoded_sequence = one_hot_encoder(sequence)
        # Append label and one-hot encoded sequence
        one_hot_encoded_sequence = np.array(one_hot_encoded_sequence)
        processed_data.append((label, one_hot_encoded_sequence))


    return processed_data

In [54]:
# Generating random data
random_train_data = np.random.rand(32,1,28, 28)
print(random_train_data.shape)
random_test_data = np.random.rand(16,1,28, 28)
print(random_test_data.shape)


# Converting the data to tensor type and floating point type
tensor_train_data = torch.from_numpy(random_train_data).float()
tensor_test_data = torch.from_numpy(random_test_data).float()
print(type(tensor_train_data))
print(type(tensor_train_data))
print()
print(tensor_train_data.shape)
print(tensor_test_data.dtype)
print(len(tensor_train_data))


# Creating random binary labels. and converting it to tensor
print()
label_test = np.random.choice([0, 1], size=len(tensor_test_data))
label_train = np.random.choice([0, 1], size=len(tensor_train_data))
print(label_train.dtype)
label_test = torch.from_numpy(label_test)
label_train = torch.from_numpy(label_train)
print(label_test.dtype)

(32, 1, 28, 28)
(16, 1, 28, 28)
<class 'torch.Tensor'>
<class 'torch.Tensor'>

torch.Size([32, 1, 28, 28])
torch.float32
32

int32
torch.int32


In [55]:

# The most important class, a custom data loader, understand how it is working.

class data_class(Dataset):
    def __init__(self,data,label):
        self.data = torch.stack([torch.tensor(d).float() for d in data])
        self.label = torch.stack([torch.tensor(l).long() for l in label])

    def __len__(self):
        return len(self.data)

    def  shape(self):
      return self.data.shape

    def __getitem__(self,id):
        data_set=self.data[id]
        labels=self.label[id]
        return data_set,labels

# calling the data_class for the raw random data
train_data=data_class(tensor_train_data,label_train)
test_data=data_class(tensor_test_data,label_test)
print('here',type(train_data))
print('here11111',train_data.shape())

print('here',type(test_data))
print('here',test_data.shape)

# Creating the data loader which is going to load the data to the AI model
train_dataloader=DataLoader(train_data,batch_size=2,shuffle=True)
test_dataloader=DataLoader(test_data,batch_size=2,shuffle=True)
print('here1',type(train_dataloader))
print('here1',type(test_dataloader))

print()


here <class '__main__.data_class'>
here11111 torch.Size([32, 1, 28, 28])
here <class '__main__.data_class'>
here <bound method data_class.shape of <__main__.data_class object at 0x0000018654DA6A50>>
here1 <class 'torch.utils.data.dataloader.DataLoader'>
here1 <class 'torch.utils.data.dataloader.DataLoader'>



C:\Users\laven\AppData\Local\Temp\ipykernel_34932\758799266.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.data = torch.stack([torch.tensor(d).float() for d in data])
C:\Users\laven\AppData\Local\Temp\ipykernel_34932\758799266.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.label = torch.stack([torch.tensor(l).long() for l in label])


In [56]:
processed_data = process_sequences('train.txt')
#print(processed_data)
d = []
l = []
for i in processed_data:
    d.append(i[1])
    l.append(i[0])

data1 = data_class(data = d, label = l)
print('here11111',data1.shape())
me=DataLoader(data1,batch_size=512,shuffle=True)
#list(train_dataloader)[0:5]
# for data,label in train_dataloader:
#     print(data, label)
#     break

print(type(me))

here11111 torch.Size([71018, 4, 195])
<class 'torch.utils.data.dataloader.DataLoader'>


In [57]:
# ########################################CUSTOM DATASET END ################################################################################
# ########################################CUSTOM DATASET END ################################################################################
# ########################################CUSTOM DATASET END ################################################################################

In [58]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [59]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class ProteinCNN(nn.Module):
#     def __init__(self, input_dim, output_dim, kernel_size=3, stride=1):
#         super(ProteinCNN, self).__init__()
#         self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=64, kernel_size=kernel_size, stride=stride)
#         self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=kernel_size, stride=stride)
#         self.pool = nn.MaxPool1d(kernel_size=2)  # Max pooling layer with kernel size 2

#         # Calculate the input size for the linear layer dynamically
#         # by passing a dummy input through the convolutional layers and pooling
#         with torch.no_grad():
#             dummy_input = torch.randn(1, input_dim, 195)  # Assuming input sequence length is 195
#             conv_output = self._forward_conv(dummy_input)
#             fc_input_size = conv_output.view(1, -1).size(1)

#         self.fc1 = nn.Linear(fc_input_size, 64)
#         self.fc2 = nn.Linear(64, output_dim)

#     def forward(self, x):
#         x = self._forward_conv(x)
#         x = F.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x

#     def _forward_conv(self, x):
#         x = F.relu(self.conv1(x))
#         x = self.pool(x)  # Apply max pooling
#         x = F.relu(self.conv2(x))
#         x = self.pool(x)  # Apply max pooling
#         x = torch.flatten(x, 1)
#         return x

# # Example usage:
# input_dim = 4  # Input dimension
# output_dim = 2  # Output dimension
# model = ProteinCNN(input_dim, output_dim)



# optimizer=optim.Adam(params=model.parameters(),lr=0.0001)
# loss_fn = nn.CrossEntropyLoss()
# # # Generate random protein sequence data
# # batch_size = 32
# # random_data = torch.randn(batch_size, input_dim, 195)  # Assuming input sequence length is 195
# # print('here',random_data.shape)
# # print('here',type(random_data))
# # # print('here',type(me[]))
# # # Forward pass
# # output = model(random_data)
# # print(output.shape)  # Check output shape


In [60]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ProteinCNN(nn.Module):
    def __init__(self, input_dim, output_dim, kernel_size=3, stride=1):
        super(ProteinCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=64, kernel_size=kernel_size, stride=stride)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=kernel_size, stride=stride)
        self.pool = nn.MaxPool1d(kernel_size=2)

        conv_output_size = self._calculate_conv_output_size(input_dim)
        self.fc1 = nn.Linear(conv_output_size, 64)
        self.fc2 = nn.Linear(64, output_dim)

    def forward(self, x):
        x = self._forward_conv(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def _forward_conv(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)  # 应用最大池化
        x = F.relu(self.conv2(x))
        x = self.pool(x)  # 应用最大池化
        x = torch.flatten(x, 1)
        return x

    def _calculate_conv_output_size(self, input_dim):
        # 根据输入维度和卷积层参数，手动计算卷积和池化后的输出大小
        with torch.no_grad():
            dummy_input = torch.randn(1, input_dim, 195)  # 假设输入序列长度为195
            conv_output = self._forward_conv(dummy_input)
            return conv_output.size(1)

# 示例用法:
input_dim = 4  # 输入维度
output_dim = 2  # 输出维度
model = ProteinCNN(input_dim, output_dim)

optimizer=optim.Adam(params=model.parameters(),lr=0.0001)
loss_fn = nn.CrossEntropyLoss()


In [61]:
# class MiniCNN(torch.nn.Module):
#   def __init__(self):
#     super(MiniCNN, self).__init__()
#     self.conv1 = torch.nn.Conv2d(1, 2, kernel_size=5)
#     self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2)
#     self.conv2 = torch.nn.Conv2d(6, 16, kernel_size=3)
#     self.fc1 = torch.nn.Linear(16 * 5 * 5, 120)
#     self.fc2 = torch.nn.Linear(120, 2)


#   def forward(self, x):
#     x = self.pool(torch.nn.functional.relu(self.conv1(x)))
#     x = self.pool(torch.nn.functional.relu(self.conv2(x)))
#     x = x.view(-1,16 * 5 * 5)
#     x = torch.nn.functional.relu(self.fc1(x))
#     x = self.fc2(x)
#     return x

# # Instantiating the model and assigning an optimizer to the model and creating a loss function
# model=MiniCNN().to(device)
# optimizer=optim.Adam(params=model.parameters(),lr=0.01)
# loss_fn = nn.CrossEntropyLoss()

In [62]:
# train_dataloader[0]
for inputs, targets in me:
    print('inp',len(inputs))
    print('tar',len(targets))

inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512
tar 512
inp 512


In [63]:
for batch_ids, (img, classes) in enumerate(me):
    print(batch_ids)
    print(img)
    print('class',len(classes))
    print()

0
tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.],
         [1., 1., 1.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 1.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         [1., 1., 1.,  ..., 1., 1., 0.]],

        [[0., 0., 0.,  ..., 0., 1., 0.],
         [1., 0., 0.,  ..., 1., 0., 0.],
         [0., 1., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 1.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [1., 1., 0.,  ..., 1., 1., 1.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 1., 1., 1.],
         [0., 1., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         [1., 1., 1.,  ..., 1.,

In [64]:
def train(model,device,train_dataloader,optimizer,epochs):
    print("inside train")
    model.train()
    for batch_ids, (img, classes) in enumerate(train_dataloader):
        # print(batch_ids)
        # print(img)
        # print('class',len(classes))
        # print()
        # classes=classes.type(torch.LongTensor)
        img,classes=img.to(device),classes.to(device)
        torch.autograd.set_detect_anomaly(True)
        optimizer.zero_grad()
        output=model(img)
        loss = loss_fn(output,classes)

        loss.backward()
        optimizer.step()
    if(batch_ids) % 2 == 0:
        print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
            epochs, batch_ids* len(img), len(train_dataloader.dataset),
            100.*batch_ids / len(train_dataloader),loss.item()))

def test(model, device, test_dataloader):
    model.eval()
    test_loss=0
    correct=0
    with torch.no_grad():
        for img,classes in test_dataloader:
            img,classes=img.to(device), classes.to(device)
            y_hat=model(img)
            test_loss+=F.nll_loss(y_hat,classes,reduction='sum').item()
            _,y_pred=torch.max(y_hat,1)
            correct+=(y_pred==classes).sum().item()
        test_loss/=len(test_dataloader)
        print("\n Test set: Avarage loss: {:.0f},Accuracy:{}/{} ({:.0f}%)\n".format(
            test_loss,correct,len(test_dataloader.dataset),100.*correct/len(test_dataloader.dataset)))
        print('='*30)

In [65]:
# WE ARE USING RANDOM DATA SO THE TRAINING AND TESTING DOES NOT MATTER, THE AIM IS TO SHOWCASE THE USE OF A CUSTOM DATASET
# SINCE IN PRACTICAL SENSE YOU HAVE TO CLEAN THE DATA AND LOAD THE DATA INTO THE MODEL.
if __name__=='__main__':
    seed=42
    EPOCHS=10
    model = model.cuda()
    for epoch in range(1,EPOCHS+1):
        train(model,device,me,optimizer,epoch)
        test(model,device,me)

inside train
Train Epoch: 1 [49956/71018 (99%)]	Loss: 0.579049

 Test set: Avarage loss: -66,Accuracy:45690/71018 (64%)

inside train
Train Epoch: 2 [49956/71018 (99%)]	Loss: 0.583598

 Test set: Avarage loss: -75,Accuracy:46961/71018 (66%)

inside train
Train Epoch: 3 [49956/71018 (99%)]	Loss: 0.600462

 Test set: Avarage loss: -87,Accuracy:47569/71018 (67%)

inside train
Train Epoch: 4 [49956/71018 (99%)]	Loss: 0.613677

 Test set: Avarage loss: -97,Accuracy:48318/71018 (68%)

inside train
Train Epoch: 5 [49956/71018 (99%)]	Loss: 0.548324

 Test set: Avarage loss: -93,Accuracy:48699/71018 (69%)

inside train
Train Epoch: 6 [49956/71018 (99%)]	Loss: 0.587912

 Test set: Avarage loss: -93,Accuracy:49812/71018 (70%)

inside train
Train Epoch: 7 [49956/71018 (99%)]	Loss: 0.595023

 Test set: Avarage loss: -104,Accuracy:49867/71018 (70%)

inside train
Train Epoch: 8 [49956/71018 (99%)]	Loss: 0.550373

 Test set: Avarage loss: -111,Accuracy:50510/71018 (71%)

inside train
Train Epoch: 9 [4